## 🔐 Prerequisites

Before running the first cell, make sure you're authenticated with Azure CLI. Run this command in your terminal:

```bash
az login
```

or

```bash
az login --use-device-code
```

# 🏦 Agent-Level and Run-Level Middleware

## Industry Use Case: Transaction Compliance Monitoring

This notebook demonstrates the difference between **agent-level** and **run-level** middleware using a **transaction compliance** scenario.

| Middleware Type | Scope | Use Cases |
|-----------------|-------|-----------|
| **Agent-Level** | Applied to ALL runs | Security validation, performance monitoring, audit logging |
| **Run-Level** | Applied to specific runs | Priority handling, debugging, caching |

### FSI Scenario
A compliance agent that monitors transactions for regulatory violations. Different middleware layers handle:
- **Security**: Block requests containing sensitive data patterns
- **Performance**: Track execution time for SLA compliance
- **Priority**: Expedite high-value transaction reviews
- **Caching**: Avoid redundant compliance checks

**Execution Order:** Agent middleware (outermost) → Run middleware (innermost) → Agent execution

In [ ]:
import os
from dotenv import load_dotenv

# Load environment variables from root .env
load_dotenv('../../.env', override=True)

PROJECT_ENDPOINT = os.environ["AI_FOUNDRY_PROJECT_ENDPOINT"]
MODEL_DEPLOYMENT = os.environ.get("AZURE_AI_MODEL_DEPLOYMENT_NAME", "gpt-4o")

print(f"✅ Project Endpoint: {PROJECT_ENDPOINT[:50]}...")
print(f"✅ Model Deployment: {MODEL_DEPLOYMENT}")

## Import Required Libraries

In [ ]:
import time
from collections.abc import Awaitable, Callable
from random import randint, choice
from typing import Annotated

from agent_framework import (
    AgentMiddleware,
    AgentResponse,
    AgentRunContext,
    FunctionInvocationContext,
)
from agent_framework.azure import AzureAIAgentClient
from azure.identity.aio import AzureCliCredential
from pydantic import Field

print("✅ Libraries imported successfully")

## Define FSI Tool Functions

Transaction compliance checking tools for the agent.

In [ ]:
def check_transaction_compliance(
    transaction_id: Annotated[str, Field(description="The transaction ID to check compliance for.")],
    amount: Annotated[float, Field(description="The transaction amount in USD.")],
) -> str:
    """Check if a transaction meets compliance requirements."""
    risk_levels = ["low", "medium", "high"]
    compliance_status = ["compliant", "needs_review", "flagged"]
    
    risk = choice(risk_levels)
    status = "flagged" if amount > 50000 else choice(compliance_status)
    
    return f"""
    Transaction Compliance Report:
    - Transaction ID: {transaction_id}
    - Amount: ${amount:,.2f}
    - Risk Level: {risk.upper()}
    - Compliance Status: {status.upper()}
    - AML Check: {'PASSED' if amount < 10000 else 'REQUIRES REVIEW'}
    - CTR Required: {'YES' if amount >= 10000 else 'NO'}
    """

def get_customer_risk_profile(
    customer_id: Annotated[str, Field(description="The customer ID to retrieve risk profile.")],
) -> str:
    """Get the risk profile for a customer."""
    risk_scores = [randint(1, 100) for _ in range(3)]
    avg_score = sum(risk_scores) / len(risk_scores)
    
    return f"""
    Customer Risk Profile:
    - Customer ID: {customer_id}
    - Risk Score: {avg_score:.0f}/100
    - Risk Category: {'HIGH' if avg_score > 70 else 'MEDIUM' if avg_score > 40 else 'LOW'}
    - Account Status: ACTIVE
    - Last Review: 2025-01-15
    """

print("✅ FSI compliance tools defined")

## Agent-Level Middleware

These middleware are applied to **ALL runs** of the agent - perfect for security, monitoring, and audit logging.

### 1. Compliance Security Middleware (Class-Based)

Blocks requests that contain sensitive PII patterns.

In [ ]:
class ComplianceSecurityMiddleware(AgentMiddleware):
    """Agent-level security middleware that validates all compliance requests."""

    async def process(
        self, 
        context: AgentRunContext, 
        next: Callable[[AgentRunContext], Awaitable[None]]
    ) -> None:
        print("[🔒 SecurityMiddleware] Validating request for compliance...")

        # Check for PII/sensitive data patterns in the request
        last_message = context.messages[-1] if context.messages else None
        if last_message and last_message.text:
            query = last_message.text.lower()
            # Block requests containing sensitive data patterns
            sensitive_patterns = ["ssn", "social security", "password", "pin number"]
            if any(pattern in query for pattern in sensitive_patterns):
                print("[🔒 SecurityMiddleware] ⚠️ PII detected! Blocking request for data protection.")
                return  # Don't call next() to prevent execution

        print("[🔒 SecurityMiddleware] ✅ Security check passed.")
        context.metadata["security_validated"] = True
        context.metadata["validated_at"] = time.strftime("%Y-%m-%d %H:%M:%S")
        await next(context)

### 2. SLA Performance Monitor (Function-Based)

Tracks execution time for regulatory SLA compliance.

In [ ]:
async def sla_performance_middleware(
    context: AgentRunContext,
    next: Callable[[AgentRunContext], Awaitable[None]],
) -> None:
    """Agent-level performance monitoring for SLA compliance."""
    print("[⏱️ SLA Monitor] Starting performance tracking...")
    start_time = time.time()

    await next(context)

    duration = time.time() - start_time
    sla_status = "✅ WITHIN SLA" if duration < 5.0 else "⚠️ SLA BREACH"
    
    print(f"[⏱️ SLA Monitor] Execution time: {duration:.3f}s - {sla_status}")
    context.metadata["execution_time"] = duration
    context.metadata["sla_compliant"] = duration < 5.0

### 3. Audit Logging Middleware

Logs all function/tool invocations for regulatory audit trail.

In [ ]:
async def audit_logging_middleware(
    context: FunctionInvocationContext,
    next: Callable[[FunctionInvocationContext], Awaitable[None]],
) -> None:
    """Function middleware that creates audit logs for all compliance tool calls."""
    function_name = context.function.name
    args = context.arguments
    timestamp = time.strftime("%Y-%m-%d %H:%M:%S")
    
    print(f"[📋 Audit Log] {timestamp} - Calling: {function_name}")
    print(f"[📋 Audit Log] Arguments: {args}")

    await next(context)

    print(f"[📋 Audit Log] {function_name} completed - logged for compliance")

## Run-Level Middleware

These middleware are applied to **specific runs only** - perfect for priority handling and caching.

### 1. High-Value Transaction Middleware

Expedites processing for high-value transactions requiring immediate review.

In [ ]:
class HighValueTransactionMiddleware(AgentMiddleware):
    """Run-level middleware for high-value transaction requests."""

    async def process(
        self, 
        context: AgentRunContext, 
        next: Callable[[AgentRunContext], Awaitable[None]]
    ) -> None:
        print("[🔴 HIGH VALUE] Processing high-value transaction with expedited handling...")

        # Verify security validation from agent-level middleware
        if context.metadata.get("security_validated"):
            print("[🔴 HIGH VALUE] Security validation confirmed from agent middleware")

        # Set high priority flags
        context.metadata["priority"] = "HIGH"
        context.metadata["expedited"] = True
        context.metadata["requires_senior_review"] = True

        await next(context)
        print("[🔴 HIGH VALUE] High-value transaction processing completed")

### 2. Debugging Middleware

Enables detailed debug logging for troubleshooting specific compliance runs.

In [ ]:
async def debugging_middleware(
    context: AgentRunContext,
    next: Callable[[AgentRunContext], Awaitable[None]],
) -> None:
    """Run-level debugging middleware for troubleshooting compliance runs."""
    print("[🔍 Debug] Debug mode enabled for this compliance check")
    print(f"[🔍 Debug] Messages count: {len(context.messages)}")
    print(f"[🔍 Debug] Is streaming: {context.is_streaming}")

    # Log existing metadata from agent middleware
    if context.metadata:
        print(f"[🔍 Debug] Agent-level metadata: {context.metadata}")

    context.metadata["debug_enabled"] = True

    await next(context)

    print("[🔍 Debug] Debug information collected")

### 3. Compliance Cache Middleware

Caches compliance check results to avoid redundant regulatory lookups.

In [ ]:
class ComplianceCacheMiddleware(AgentMiddleware):
    """Run-level caching middleware for compliance check results."""

    def __init__(self) -> None:
        self.cache: dict[str, AgentResponse] = {}

    async def process(
        self, 
        context: AgentRunContext, 
        next: Callable[[AgentRunContext], Awaitable[None]]
    ) -> None:
        # Create cache key from the last message
        last_message = context.messages[-1] if context.messages else None
        cache_key: str = last_message.text if last_message and last_message.text else "no_message"

        if cache_key in self.cache:
            print(f"[💾 Cache] ✅ Cache HIT for: '{cache_key[:40]}...'")
            context.result = self.cache[cache_key]
            return  # Don't call next(), return cached result

        print(f"[💾 Cache] ❌ Cache MISS for: '{cache_key[:40]}...'")
        context.metadata["cache_key"] = cache_key

        await next(context)

        # Cache the result
        if context.result:
            self.cache[cache_key] = context.result
            print("[💾 Cache] Result cached for future compliance checks")

## Run the Compliance Agent Demo 🚀

This demonstrates:
1. Agent with **agent-level middleware** (security, SLA monitoring, audit logging)
2. Multiple runs with different **run-level middleware** configurations
3. How agent and run middleware interact in a compliance scenario

In [ ]:
async def main() -> None:
    """Demonstrate agent-level and run-level middleware for compliance."""
    print("=" * 70)
    print("🏦 TRANSACTION COMPLIANCE MIDDLEWARE DEMO")
    print("=" * 70)
    print()

    async with (
        AzureCliCredential() as credential,
        AzureAIAgentClient(
            project_endpoint=PROJECT_ENDPOINT,
            AZURE_AI_MODEL_DEPLOYMENT_NAME=MODEL_DEPLOYMENT,
            credential=credential,
        ).as_agent(
            name="ComplianceAgent",
            instructions="""You are a transaction compliance assistant for a financial institution.
            Help users check transaction compliance, review customer risk profiles, and ensure
            regulatory requirements are met. Always be thorough and cite specific compliance rules.""",
            tools=[check_transaction_compliance, get_customer_risk_profile],
            # Agent-level middleware: applied to ALL runs
            middleware=[
                ComplianceSecurityMiddleware(),
                sla_performance_middleware,
                audit_logging_middleware,
            ],
        ) as agent,
    ):
        print("✅ Agent created with agent-level middleware:")
        print("   - ComplianceSecurityMiddleware (blocks PII)")
        print("   - SLA Performance Monitor (tracks execution time)")
        print("   - Audit Logging (logs all tool calls)")
        print()

        # Run 1: Standard compliance check (agent-level middleware only)
        print("=" * 70)
        print("RUN 1: Standard Compliance Check (agent-level middleware only)")
        print("=" * 70)
        query = "Check compliance for transaction TXN-2025-001 with amount $5,000"
        print(f"👤 User: {query}")
        result = await agent.run(query)
        print(f"🤖 Agent: {result.text if result.text else 'No response'}")
        print()

        # Run 2: High-value transaction (add run-level priority middleware)
        print("=" * 70)
        print("RUN 2: High-Value Transaction (+ run-level priority middleware)")
        print("=" * 70)
        query = "URGENT: Check compliance for transaction TXN-2025-002 with amount $75,000"
        print(f"👤 User: {query}")
        result = await agent.run(
            query,
            middleware=[HighValueTransactionMiddleware()],
        )
        print(f"🤖 Agent: {result.text if result.text else 'No response'}")
        print()

        # Run 3: Debug mode for troubleshooting
        print("=" * 70)
        print("RUN 3: Debug Mode (+ run-level debugging middleware)")
        print("=" * 70)
        query = "Get risk profile for customer CUST-98765"
        print(f"👤 User: {query}")
        result = await agent.run(
            query,
            middleware=[debugging_middleware],
        )
        print(f"🤖 Agent: {result.text if result.text else 'No response'}")
        print()

        # Run 4: Caching test - first call
        print("=" * 70)
        print("RUN 4: Caching Test - First Call (+ cache middleware)")
        print("=" * 70)
        cache_middleware = ComplianceCacheMiddleware()
        query = "Check compliance for transaction TXN-2025-003 with amount $12,500"
        print(f"👤 User: {query}")
        result = await agent.run(
            query,
            middleware=[cache_middleware, debugging_middleware],
        )
        print(f"🤖 Agent: {result.text if result.text else 'No response'}")
        print()

        # Run 5: Caching test - cache hit (same query)
        print("=" * 70)
        print("RUN 5: Caching Test - Cache Hit (same query as Run 4)")
        print("=" * 70)
        print(f"👤 User: {query}")  # Same query
        result = await agent.run(
            query,
            middleware=[cache_middleware],  # Same cache instance
        )
        print(f"🤖 Agent: {result.text if result.text else 'No response'}")
        print()

        # Run 6: Security test (should be blocked)
        print("=" * 70)
        print("RUN 6: Security Test (should be BLOCKED by agent middleware)")
        print("=" * 70)
        query = "Check compliance for SSN 123-45-6789"
        print(f"👤 User: {query}")
        result = await agent.run(query)
        print(f"🤖 Agent: {result.text if result.text else '⛔ Request blocked by security middleware'}")
        print()

        print("=" * 70)
        print("✅ DEMO COMPLETE")
        print("=" * 70)

# Run the demo
await main()

## Key Takeaways 📚

### Agent-Level vs Run-Level Middleware

| Aspect | Agent-Level | Run-Level |
|--------|-------------|-----------|
| **Scope** | ALL runs | Specific runs only |
| **When Configured** | Agent creation | Per run |
| **Industry Use Cases** | Security, SLA, Audit | Priority, Debug, Cache |
| **Persistence** | Always active | Request-specific |

### Execution Order

```
Agent Middleware → Run Middleware → Agent Execution → Run Middleware → Agent Middleware
    (outermost)      (innermost)                        (innermost)      (outermost)
```

### FSI Compliance Patterns

| Pattern | Middleware Type | Purpose |
|---------|-----------------|---------|
| **PII Detection** | Agent-level | Block sensitive data in all requests |
| **SLA Monitoring** | Agent-level | Track performance for compliance |
| **Audit Trail** | Agent-level | Log all operations for regulators |
| **Priority Queue** | Run-level | Expedite high-value transactions |
| **Response Cache** | Run-level | Avoid redundant compliance checks |